In [1]:
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

train = pd.read_csv('../input/birdclef-2021/train_metadata.csv',)
train.head(2)

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,date,filename,license,rating,time,url
0,acafly,['amegfi'],"['begging call', 'call', 'juvenile']",35.3860,-84.1250,Empidonax virescens,Acadian Flycatcher,Mike Nelson,2012-08-12,XC109605.ogg,Creative Commons Attribution-NonCommercial-Sha...,2.5,09:30,https://www.xeno-canto.org/109605
1,acafly,[],['call'],9.1334,-79.6501,Empidonax virescens,Acadian Flycatcher,Allen T. Chartier,2000-12-26,XC11209.ogg,Creative Commons Attribution-NonCommercial-Sha...,3.0,?,https://www.xeno-canto.org/11209


In [2]:
values_counts = train['primary_label'].value_counts()
values_counts

primary_label
redcro     500
comrav     500
houspa     500
norcar     500
barswa     500
          ... 
whcpar      12
wegspa1     10
runwre1      9
crfpar       8
stvhum2      8
Name: count, Length: 397, dtype: int64

In [3]:
unique_values_count = len(train['primary_label'].value_counts())
print(unique_values_count,'\n')
top_10_labels = train['primary_label'].value_counts().head(10)
print(top_10_labels)


397 

primary_label
redcro     500
comrav     500
houspa     500
norcar     500
barswa     500
houwre     500
sonspa     500
gbwwre1    500
eursta     500
spotow     500
Name: count, dtype: int64


In [4]:
top_10_labels = train['primary_label'].value_counts().index[0:10]
print(top_10_labels)


Index(['redcro', 'comrav', 'houspa', 'norcar', 'barswa', 'houwre', 'sonspa',
       'gbwwre1', 'eursta', 'spotow'],
      dtype='object', name='primary_label')


In [ ]:
import os
directory_names = []

for dirname, _, _ in os.walk('/kaggle/input/birdclef-2021/train_short_audio'):
    # Check if the current directory is related to one of the top 10 bird labels
    if any(label in dirname for label in top_10_labels):
        directory_names.append(dirname)

# Print the stored directory names
print("Directory names related to 10 most occuring bird labels:")
for directory in directory_names:
    print(directory)


In [8]:
bird_label_files = {}

for label_dir in directory_names:
    label = label_dir.split('/')[-1]  # Extract label from directory path
    bird_label_files[label] = []  # Initialize list for filenames
    for _, _, filenames in os.walk(label_dir):
        for filename in filenames:
            bird_label_files[label].append(os.path.join(label_dir, filename))

# Print the dictionary containing filenames for each bird label
# Print the number of files under each label
for label, files in bird_label_files.items():
    print(f"Number of files under {label}: {len(files)}")


Number of files under eursta: 500
Number of files under houwre: 500
Number of files under norcar: 500
Number of files under houspa: 500
Number of files under sonspa: 500
Number of files under comrav: 500
Number of files under redcro: 500
Number of files under gbwwre1: 500
Number of files under spotow: 500
Number of files under barswa: 500


In [ ]:

directory = "/kaggle/input/birdclef-2021/train_short_audio/"

# Get list of all subdirectories
subdirectories = [subdir for subdir in os.listdir(directory) if os.path.isdir(os.path.join(directory, subdir))]

len(subdirectories)


In [ ]:
directory = "/kaggle/input/birdclef-2021/train_short_audio/"

subdirectories = [subdir for subdir in os.listdir(directory) if os.path.isdir(os.path.join(directory, subdir))]

for label in bird_label_files.keys():
    if label in subdirectories:
        subdirectories.remove(label)

print(len(subdirectories))


In [ ]:

from tqdm import tqdm

base_dir = '/kaggle/input/birdclef-2021/train_short_audio/'

for bird_label in bird_label_files.keys():
    label_dir = os.path.join(base_dir, bird_label)
    zip_file_name = bird_label + ".zip"
    
    # Count the number of files in the directory
    num_files = sum([len(files) for _, _, files in os.walk(label_dir)])
    
    # Create a zip file with a progress bar
    with tqdm(total=num_files, desc=f"Zipping {bird_label} files") as pbar:
        !zip -r {zip_file_name} {label_dir} --quiet
        pbar.update(num_files)




In [5]:
import os
import random
import zipfile
from tqdm import tqdm

# Base directory for audio files
base_dir = "/kaggle/input/birdclef-2021/train_short_audio/"

# Get list of non-top-10 species directories
all_species = [d for d in os.listdir(base_dir) 
               if os.path.isdir(os.path.join(base_dir, d))]
non_top_species = [d for d in all_species if d not in top_5_labels]

# Collect ALL files from non-top species
all_files = []
for species in non_top_species:
    species_path = os.path.join(base_dir, species)
    for root, _, files in os.walk(species_path):
        all_files.extend([os.path.join(root, f) for f in files])

# Randomly select 500 files total (or max available)
random.seed(42)
max_samples = min(500, len(all_files))
selected_files = random.sample(all_files, max_samples)

# Create zip with preserved directory structure
with zipfile.ZipFile('others.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file_path in tqdm(selected_files, desc="Zipping others"):
        # Preserve relative path from base directory
        arcname = os.path.relpath(file_path, start=base_dir)
        zipf.write(file_path, arcname)

print(f"\nCreated others.zip with {len(selected_files)} files from {len(non_top_species)} species")

Zipping others: 100%|██████████| 500/500 [00:22<00:00, 22.49it/s]


Created others.zip with 500 files from 387 species


In [6]:
# Check zip contents
from collections import defaultdict
import zipfile

species_count = defaultdict(int)

with zipfile.ZipFile('others.zip', 'r') as zipf:
    for f in zipf.namelist():
        species = f.split('/')[0]
        species_count[species] += 1

print(f"Total files: {sum(species_count.values())}")
print(f"Species represented: {len(species_count)}")
print("Sample species distribution:")
for species, count in list(species_count.items())[:5]:
    print(f"{species}: {count} files")

Total files: 500
Species represented: 261
Sample species distribution:
ovenbi1: 4 files
littin1: 3 files
rufhum: 1 files
larspa: 2 files
rebwoo: 1 files
